[View in Colaboratory](https://colab.research.google.com/github/thiagojoa/estudo_python/blob/master/Crawler_Scraping.ipynb)

## Crawler e Scraping para captura de dados na WEB

Neste notebook, vou mostrar passo a passo como obter dados de um produto que esta sendo vendido em uma página web (E-commerce)

Antes gostaria de dizer que sou entusiasta Python e ainda estou aprendendo a utiliza-lo, em resumo quero dizer que todas as sugestões, correções entre outras serão bem vindas, esntão desde já agradeço seu comentário e/ou correções.

Meu e-mail para caso tenham dúvidas é [thiago.msbueno@gmail.com](https://)


Para executarmos esta extração de dados, utilizaremos Python com as bibliotecas, [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/), [Requests](http://docs.python-requests.org/pt_BR/latest/), e [Re](https://docs.python.org/2/library/re.html) (Lib regex para manipularmos algumas strings)

##Como definir a estratégia para obter dados de uma página web

Eu particularmente dividirei este processo de scraping nas seguintes etapas:

1 - Compreensão dos alvos, isto é, em qual página web esta localizada as informações que queremos.

2 - Localização dos alvos, isto é, onde estão os dados que quermos obter dentro da estrutura html da página.

3 - Identificação do método de extração, por exemplo, o alvo esta dentro de tags html? Dentro de tags html de script em um json? Como e onde o alvo está dentro do codigo?

4- Obtenção dos dados com as ferramentas escolhidas.

Neste exemplo usaremos uma Churrasqueira Nautika, que está a venda em um e-commerce, onde queremos obter os seguintes dados referente ao produto:

Titulo da Pagina do produto:
Descrição do produto:
Preço:
Marca:
Categoria:
SKU:

**Informações adicionais:**

*E-commerce escolhido para exemplo: https://www.lojasestrela.com.br*

*Produto escolhido para obtenção de dados: Churrasqueira Nautika Colorado*

*Link da pagina do produto: https://www.lojasestrela.com.br/churrasqueira-nautika-colorado*


##1 - Compreensão do Alvo
Sabemos que o nosso objetivo é capturar os dados de preço, marca, descrição, titulo da página e SKU da Churrasqueira Nautika que é um produto ativo do e-commerce da Empresa [Lojas Estrela](https://www.lojasestrela.com.br), sendo assim, vamos navegar na pagina do e-commerce e localizar este produto. 

Após localizarmos o produto, achamos o link do mesmo, que é https://www.lojasestrela.com.br/churrasqueira-nautika-colorado , sendo assim, já podemos pular para próxima etapa que é a parte de analíse do HTML desta página.

##2 - Localização dos alvos
Sabemos que os alvos que quermos  encontrar dentro deste html, são códigos que nos retornem os dados referentes ao titulo da Pagina do produto, descrição do produto, preço, marca, categoria e SKU. Sendo assim abaixo temos o HTML completo da página obtido através do inspetor de elemtnos do navegador, normalmente Ctrl + Shift + i no navegador, na página em questão. 

Abaixo faremos uma análise neste HTML para verificar se encontraremos os alvos dentro dele, segue HTML abaixo, para abri-lo basta clicar em SHOW CODE

##3 - Identificação do método de extração
Neste passo, após analisar o HTML acima, conclui-se que todos os dados que temos como objetivo capturar, estão no HTML referido e que este e-commerce carrega grande parte dos dados dos pordutos, dentro de um JSON localizado dentro de uma tag script no head do HTML em questão. O Codigo abaixo vem com alguns erros de charset, mas é perfeitamente interpretado pelo Python.
```

<script>dataLayer = [{"pageTitle":"Churrasqueira Nautika Colorado Nautika nas Lojas Estrela - M\u00f3veis, Eletrodom\u00e9sticos, Foto \u00d3tica, Departamentos e muito mais","pageCategory":"Produto","event":"","siteSearchFrom":"","idProduct":"7398","nameProduct":"Churrasqueira Nautika Colorado","category":"Churrasqueiras","idCategory":"73","priceSell":"436.00","promotion":"NO","price":"436.00","brand":"Nautika","reference":"7398","model":"","availability":"YES","availabilityDetails":"Imediata","urlImage":"https:\/\/images.tcdn.com.br\/img\/img_prod\/587522\/churrasqueira_nautika_colorado_7398_1_20170823161515.jpg","urlProduct":"http:\/\/www.lojasestrela.com.br\/churrasqueira-nautika-colorado","listSku":[],"characteristcs":[],"priceSellDetails":[{"name":"","installment.months":"1","installment.amount":"414.20"},{"name":"","installment.months":"10","installment.amount":"43.60"}],"EAN":"","breadcrumbDetails":[{"id":"41","name":"Esporte e Lazer","level":1},{"id":"73","name":"Churrasqueiras","level":2}],"freeShipping":"NO","hot":"NO","additionalButton":"NO","release":"NO","rating":{"count":"0","average":"0"},"breadcrumb":"P\u00e1gina Inicial > Esporte e Lazer > Churrasqueiras"}]</script><script>
                    var gtmTray = document.createElement("script");
                    gtmTray.src = "/mvc/store/587522/google_tag_manager/updateGTM.js?" + Date.now();
                    document.querySelector("head").appendChild(gtmTray);
                </script>
```

**Sendo assim...**
Temos então que nossos dados estão distribuidos da seguinte maneira dentro de nosso código:

Preço -> Dentro do JSON na no script da tag HEAD
Marca -> Dentro do JSON na no script da tag HEAD
Descrição -> Dentro do JSON na no script da tag HEAD
Titulo da página -> Na tag TITLE que se localiza na tag HEAD
SKU -> Dentro do JSON na no script da tag HEAD

Enfim, tirando o titulo da página, todos os nossos alvos estão dentro do JSON, e as minhas escolhas são: 

Para os alvos que estão no JSON

Preço, Marca, Descrição e SKU -> Utilizar a extração via beatifulsoup do head do documento e após utilizar REGEX para extrair dados exatos. 

Já para o titulo da página, podemos utilizar apenas a lib beatifulsou, pois ele se localiza na tag title. 

Tudo definido, partimos para extração dos dados com Python e suas bibliotecas no passo 4



##Obtenção dos dados com as ferramentas escolhidas
Agora seguimos com a programação python que faz a extração dos dados, explicação do código estarão como comentários nos blocos abaixo, para testar o código deve-se executar bloco a bloco consecutivamente


In [2]:
#este pip instala a lib lxml no google colab
!pip install lxml

#Importando todas as bibliotecas que usaremos no ambiente
from bs4 import BeautifulSoup as bs
import requests
import re
import lxml

#Agora colocaremos na variavel html o link da pagina de produto que vamos
#utilizar, você pode escolher outro prduto e colocar aqui para testar

html = 'https://www.lojasestrela.com.br/churrasqueira-nautika-colorado'

#html também será a variavel que vai armazenar o conteúdo html da pagina
html = requests.get(html)

#Gerando o objeto BeautifulSoup com o conteudo da variavel html
soup = bs(html.content, "lxml")

#agora, convertemos o o objeto retornado em string para utilizarmos com o regex
string_html = str(soup.html.head)

#print(soup.html.head) #teremos a saida do head do html

#Agora faremos a busca via regex dos dados alvo
#Foi verificado que o alvo esta dentro do trecho de Codigo
#'<script>dataLayer=......' sendo assim o codigo regex que da
#match no preço por exemplo é ""priceSell":"(.\w.\w*[.]\w*)/"
#voce pode checar essa regex no site https://www.regexpal.com/

#Capturando o preço do produto com regex

#capturando o titulo da pagina, sabendo que ja temos ele no objeto
#Soup, podendo então capturalo através do HTML -> HEAD - > TITLE
#porem vamos utilizar o find para ser mais um exemplo
titulo_pagina = soup.find('title')

#Agora transformar a var em string para remover tags html e utilização posterior
titulo_pagina = str(titulo_pagina)

#Cria varial com codigo regex que remove tags html de strings
TAG_RE = re.compile(r'<[^>]+>')

#cria-se função que remove tags html de strings
def remove_tags(text):
    return TAG_RE.sub('', text)

#remove as tags html chamando a funcao que remove tags e pronto
#nossa variavel titulo_pagina agora é uma string sem tags htmls
titulo_pagina = remove_tags(titulo_pagina)

#capturando a descricao do produtos com find regex
descricao = re.findall(r'(?:"nameProduct":")(.*?)(?:")', string_html)
#Agora transformar a var em string para utilização posterior
descricao = str(descricao)

#capturando preço do produto com find regex
preco = re.findall(r'"priceSell":"(.\w*[.]\w*)', string_html)
#Agora transformar a var em string para utilização posterior
preco = str(preco)

#capturando a marca do produto com find regex
marca = re.findall(r'(?:"brand":")(.*?)(?:")', string_html)
#Agora transformar a var em string para utilização posterior
marca = str(marca)

#capturando a categoria do produto com find regex
categoria = re.findall(r'(?:"brand":")(.*?)(?:")', string_html)
#Agora transformar a var em string para utilização posterior
categoria = str(categoria)

#capturando o SKU do produto com find regex
sku= re.findall(r'(?:"idProduct":")(.*?)(?:")', string_html)
#Agora transformar a var em string para utilização posterior
sku = str(sku)

print('Título da Página: '+ titulo_pagina)
print('Descrição do Produto: '+ descricao)
print('Preço: '+ preco)
print('Marca: '+ marca)
print('Sku: '+sku)


Título da Página: Churrasqueira Nautika Colorado Nautika nas Lojas Estrela - Móveis, Eletrodomésticos, Foto Ótica, Departamentos e muito mais
Descrição do Produto: ['Churrasqueira Nautika Colorado']
Preço: ['436.00']
Marca: ['Nautika']
Sku: ['7398']
